<a href="https://colab.research.google.com/github/rami2ee3/DA_study/blob/main/eviot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from os import path
import pandas as pd

path = "/content/drive/MyDrive/Colab Notebooks/EV_Predictive_Maintenance_Dataset_15min.csv"

df = pd.read_csv(path, header=1, encoding="cp949")

print(df.head())
print(df.columns[:10])
print(df.shape)

/tmp/ipython-input-2464657918.py:6: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=1, encoding="cp949")


         Timestamp       SoC       SoH  Battery_Voltage  Battery_Current  \
0  2020-01-01 0:00  0.826099  0.941338       210.163881       -22.753095   
1  2020-01-01 0:15  0.064728  0.916059       364.000102       -27.701120   
2  2020-01-01 0:30  0.873643  0.908020       388.855089       -36.646406   
3  2020-01-01 0:45  0.853009  0.916476       370.570602       -37.609429   
4  2020-01-01 1:00  0.947540  0.913206       390.011904       -14.275808   

   Battery_Temperature  Charge_Cycles  Motor_Temperature  Motor_Vibration  \
0            27.149201     149.190930          48.496049         0.369095   
1            53.655101     171.702388          57.829492         1.449195   
2            29.559090     191.617645          46.518363         1.859045   
3            29.690283     111.881817          54.163681         0.381500   
4            28.864338     163.774377          42.075978         0.433927   

   Motor_Torque  ...  Load_Weight  Driving_Speed  Distance_Traveled  \
0    113.

In [17]:
df.dtypes

,0
Timestamp,object
SoC,float64
SoH,float64
Battery_Voltage,float64
Battery_Current,float64
Battery_Temperature,float64
Charge_Cycles,float64
Motor_Temperature,float64
Motor_Vibration,float64
Motor_Torque,float64


In [18]:
df.isnull().sum()

,0
Timestamp,0
SoC,0
SoH,0
Battery_Voltage,0
Battery_Current,0
Battery_Temperature,0
Charge_Cycles,0
Motor_Temperature,0
Motor_Vibration,0
Motor_Torque,0


In [22]:
from sklearn.preprocessing import StandardScaler

target = "RUL"    # 예측 타깃 변수 : RUL(잔여수명)

# 입력(X), 출력(Y) 분리
X = df.drop(columns=["Timestamp", "RUL", "Failure_Probability", "Maintenance_Type", "TTF", "Component_Health_Score"])
y = df[target]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # fit : X의 평균과 표준편차 계산
                                    # transform : 각 값에서 평균을 빼고 표준편차로 나눔

X_scaled = pd.DataFrame(X_scaled, columns=X.columns)




In [ ]:
df = df.sort_values("Timestamp").reset_index(drop=True)

In [26]:
print(X_scaled.head())

        SoC       SoH  Battery_Voltage  Battery_Current  Battery_Temperature  \
0  0.158464  0.358726        -2.580253         0.554241            -0.722129   
1 -2.452015  0.205333         0.204296         0.445154             2.340308   
2  0.321474  0.156556         0.654190         0.247939            -0.443695   
3  0.250728  0.207867         0.323227         0.226708            -0.428537   
4  0.574842  0.188021         0.675129         0.741138            -0.523965   

   Charge_Cycles  Motor_Temperature  Motor_Vibration  Motor_Torque  Motor_RPM  \
0      -0.416363          -0.487548        -0.356501     -0.864316  -0.201575   
1      -0.279520           0.116665         2.130199     -0.966554  -0.367929   
2      -0.158459          -0.615576         3.073794     -0.783882  -0.298024   
3      -0.643157          -0.120646        -0.327942      0.035813  -0.540313   
4      -0.327713          -0.903160        -0.207241     -0.084520  -0.546691   

   ...  Tire_Pressure  Tire_Temp

In [36]:
split = int(len(X_scaled)*0.7)
X_tr, X_te = X_scaled.iloc[:split], X_scaled.iloc[split:]  # train input
y_tr, y_te = y.iloc[:split], y.iloc[split:]  # test input

In [37]:
import numpy as np

def make_windows(X_df, y_sr, time_steps=8, horizon=1, stride=3):
  Xv, yv = X_df.values, y_sr.values +
  T, F = Xv.shape
  last = T - time_steps - horizon + 1

  X_seq, y_seq = [], []
  for start in range (0, last, stride):
    end = start + time_steps
    X_seq.append(Xv[start:end, :])
    y_seq.append(yv[end + horizon -1])
  return np.stack(X_seq), np.array(y_seq)

time_steps, horizon = 16, 1
Xtr_seq, ytr_seq = make_windows(X_tr, y_tr, time_steps, horizon)
Xte_seq, yte_seq = make_windows(X_te, y_te, time_steps, horizon)

print(Xtr_seq.shape, ytr_seq.shape)


# 과거 time_steps 만큼의 센서 기록 → horizon 뒤의 타깃 예측


(122759, 16, 24) (122759,)


In [40]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

n_features = Xtr_seq.shape[2]

model = models.Sequential([
    layers.Input(shape=(time_steps, n_features)),
    layers.LSTM(32),
    layers.Dense(32, activation="relu"),
    layers.Dense(1)
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

es = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
rlr = callbacks.ReduceLROnPlateau(patience=3, factor=0.5)

history = model.fit(
    Xtr_seq, ytr_seq,
    validation_data=(Xte_seq, yte_seq),
    epochs=10, batch_size=256,
    callbacks=[es, rlr],  # 과적합 방지, 학습률 조절
    verbose=1, # 학습 로그 출력
    shuffle=False
)


Epoch 1/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - loss: 38910.6172 - mae: 178.9481 - val_loss: 7154.9722 - val_mae: 62.3500 - learning_rate: 0.0010
Epoch 2/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - loss: 7196.2324 - mae: 60.5981 - val_loss: 7108.2129 - val_mae: 59.3093 - learning_rate: 0.0010
Epoch 3/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - loss: 7187.2246 - mae: 59.6571 - val_loss: 7108.5479 - val_mae: 59.4133 - learning_rate: 0.0010
Epoch 4/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - loss: 7187.4077 - mae: 59.6750 - val_loss: 7108.9517 - val_mae: 59.5073 - learning_rate: 0.0010
Epoch 5/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - loss: 7187.6167 - mae: 59.6847 - val_loss: 7109.3223 - val_mae: 59.5801 - learning_rate: 0.0010
Epoch 6/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - loss: 7187.3516 - mae: 59.7167 - val_loss: 7108.5850 - val_mae: 59.4232 - learning_rate: 5.0000e-04
Epoch 7/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - loss: 7187.3389 - mae: 59.6766 

In [41]:

test_loss, test_mae = model.evaluate(Xte_seq, yte_seq, verbose=0)
print("Test MSE:", test_loss, "Test MAE:", test_mae)

y_pred = model.predict(Xte_seq).ravel()


Test MSE: 7108.21875 Test MAE: 59.3093147277832
1644/1644 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step
